In [ ]:
import numpy as np
import h5py
from scipy import signal
from scipy import fft

In [ ]:
def get_stroke_indices(stroke_cutoffs):
  '''
  Returns a list of timestamp indices indicating the beginning of a new stroke

    Parameters:
        stroke_cutoffs (list): List of 1's and 0's indicating whether a stroke has ended at the timestamp at its index
    
    Returns:
        indices (list): List of integer indices naming the indices at which a new stroke is initiated
  '''
  # Find  all indices of stroke_cutoff where the value is a 1
  indices = (np.where(stroke_cutoffs == 1))[0]
  # 0 index is always the start of a stroke
  indices.insert(0, 0)
  return indices

In [ ]:
def extract_kinematics(drill_pose, timestamps, stroke_indices):
  '''
  Returns mean, median, and max velocity values across all strokes from drill pose data

    Parameters:
      drill_pose (list): Drill pose data directly extracted from hdf5 file
      timestamps (list): Timestamp data directly extracted from hdf5 file
      stroke_indices (list): List of integer indices naming indices of timestamps at which a new stroke is initiated

    Returns: 
      velocities (list): List containing mean, median, and max velocity
      accelerations (list): List containing mean, median, and max acceleration
  '''

  # Extract x, y, and z data from drill pose data
  x = [i[0] for i in drill_pose]
  y = [i[1] for i in drill_pose]
  z = [i[2] for i in drill_pose]

  stroke_vx = []
  stroke_vy = []
  stroke_vz = []
  stroke_vt = []
  stroke_velocities = []
  # Store velocity information for acceleration and calculate average velocity
  for i in range(len(stroke_indices) - 1):
    stroke_start = stroke_indices[i]
    next_stroke = stroke_indices[i+1]
    # Split up positional data for each stroke
    stroke_x = x[stroke_start:next_stroke]
    stroke_y = y[stroke_start:next_stroke]
    stroke_z = z[stroke_start:next_stroke]
    stroke_t = timestamps[stroke_start:next_stroke]
    # Calculate numerical derivatives between successive timestamps
    v_x = np.diff(stroke_x) / np.diff(stroke_t)
    v_y = np.diff(stroke_y) / np.diff(stroke_t)
    v_z = np.diff(stroke_z) / np.diff(stroke_t)
    stroke_vx.append(v_x)
    stroke_vy.append(v_y)
    stroke_vz.append(v_z)
    # Calculate times at which derivatives are calculated
    v_t = [(stroke_t[j + 1] + stroke_t[j]) / 2 for j in range(len(v_x))]
    stroke_vt.append(v_t)
    # Calculate distance traveled during stroke and use to calculate velocity
    curr_stroke = [[stroke_x[k], stroke_y[k], stroke_z[k]] for k in range(len(stroke_x))]
    dist = 0
    for l in range(1, len(curr_stroke)):
      dist += np.linalg.norm(curr_stroke[l] - curr_stroke[l - 1])
    stroke_velocities.append(dist / np.ptp(stroke_t))

  stroke_accelerations = []
  # Calculate average acceleration using velocity information
  for i in range(len(stroke_vx)):
    curr_stroke = [[stroke_vx[i][j], stroke_vy[i][j], stroke_vz[i][j]] for j in range(len(stroke_vx[i]))]
    vel = 0
    for k in range(1, len(curr_stroke)):
      vel += np.linalg.norm(curr_stroke[k] - curr_stroke[k - 1])
    stroke_acclerations.append(vel / np.ptp(stroke_vt[i]))
  
  # Return mean, median, and max kinematic values
  velocities = []
  accelerations = []
  velocities.append(np.mean(stroke_velocities))
  velocities.append(np.median(stroke_velocities))
  velocities.append(np.max(stroke_velocities))
  accelerations.append(np.mean(stroke_accelerations))
  accelerations.append(np.median(stroke_accelerations))
  accelerations.append(np.max(stroke_accelerations))
  return velocities, accelerations

In [ ]:
def preprocess(drill_pose):
  '''
  Applies second order Butterworth filter and FFT for curve smoothing and pattern repetition isolation

    Parameters:
      drill_pose(list): Drill pose data directly extracted from hdf5 file

    Returns:
      x (list): Preprocessed x position data
      y (list): Preprocessed y position data
      z (list): Preprocessed z position data
  '''
  # Extract x, y, and z data from drill pose data
  x_raw = [i[0] for i in drill_pose]
  y_raw = [i[1] for i in drill_pose]
  z_raw = [i[2] for i in drill_pose]

  # Smooth data using a second order Butterworth filter
  b, a = signal.butter(2, [0.005, 1], 'bandpass', analog=False)
  x_filt = signal.filtfilt(b, a, x_raw)
  y_filt = signal.filtfilt(b, a, y_raw)
  z_filt = signal.filtfilt(b, a, z_raw)

  # Apply a FFT to data
  x = fft(x_filt)
  y = fft(y_filt)
  z = fft(zfilt)

  return x, y, z
  

In [ ]:
def extract_jerk(drill_pose, timestamps, stroke_indices):
  '''
  Returns mean, median, and max jerk across all strokes from drill pose data

    Parameters:
      drill_pose (list): Drill pose data directly extracted from hdf5 file
      timestamps (list): Timestamp data directly extracted from hdf5 file
      stroke_indices (list): List of integer indices naming indices of timestamps at which a new stroke is initiated

    Returns: 
      jerks (list): List containing mean, median, and max jerk
  '''
  # Get preprocessed, x, y, and z position data
  x, y, z = preprocess(drill_pose)

  stroke_vx = []
  stroke_vy = []
  stroke_vz = []
  stroke_vt = []
  # Store velocity information for acceleration
  for i in range(len(stroke_indices) - 1):
    stroke_start = stroke_indices[i]
    next_stroke = stroke_indices[i+1]
    # Split up positional data for each stroke
    stroke_x = x[stroke_start:next_stroke]
    stroke_y = y[stroke_start:next_stroke]
    stroke_z = z[stroke_start:next_stroke]
    stroke_t = timestamps[stroke_start:next_stroke]
    # Calculate numerical derivatives between successtive timestamps
    v_x = np.diff(stroke_x) / np.diff(stroke_t)
    v_y = np.diff(stroke_y) / np.diff(stroke_t)
    v_z = np.diff(stroke_z) / np.diff(stroke_t)
    stroke_vx.append(v_x)
    stroke_vy.append(v_y)
    stroke_vz.append(v_z)
    # Calculate times at which derivatives are calculated
    v_t = [(stroke_t[j + 1] + stroke_t[j]) / 2 for j in range(len(v_x))]
    stroke_vt.append(v_t)

  stroke_ax = []
  stroke_ay = []
  stroke_az = []
  stroke_at = []
  stroke_accelerations = []
  # Store acceleration information for jerk 
  for i in range(len(stroke_vx)):
    # Calculate numerical derivatives between successive timestamps
    s_ax = np.diff(stroke_vx) / np.diff(stroke_vt)
    s_ay = np.diff(stroke_vy) / np.diff(stroke_vt)
    s_az = np.diff(stroke_vz) / np.diff(stroke_vt)
    stroke_ax.append(s_ax)
    stroke_ay.append(s_ay)
    stroke_az.append(s_az)
    # Calculate times at which derivatives are calculated
    a_t = [(stroke_vt[j + 1] + stroke_vt[j] / 2 for j in range(len(s_ax)))]
    stroke_at.append(a_t)

  stroke_jerks = []
  # Calculate average jerk using acceleration information
  for i in range(len(stroke_ax)):
    curr_stroke = [[stroke_ax[i][j], stroke_ay[i][j], stroke_az[i][j]] for j in range(len(stroke_ax[i]))]
    jerk = 0
    for k in range(1, len(curr_stroke)):
      jerk += np.linalg.norm(curr_stroke[k] - curr_stroke[k - 1])
    stroke_acclerations = []

  
  # Return mean, median, and max jerk values
  jerks = []
  np.appen
  
  return velocities, accelerations


In [ ]:
a = [2, 6, 10]
b = [1, 3, 5]
c = [[a[i], b[i]] for i in range(len(a))]
print(c)

[[2, 1], [6, 3], [10, 5]]
